In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Credits : #https://www.kaggle.com/rtatman/sql-scavenger-hunt-handbook/notebook 

1- SQL is an important skill for every data scientist , it comes in third place as most popular software tool for data science. 
2-BigQuery is a Google Cloud product for storing and accessing very large databases very quickly 

In [2]:
# import our bq_helper package
import bq_helper
# create a helper object for our bigquery dataset
hacker_news = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "hacker_news")

Using Kaggle's public dataset BigQuery integration.


Now that we've created our helper object, we can get started actually interacting with the dataset!

Now we want to check out hte way the data is structured. we will look at the schema which is the description of how data is organized  within a dataset. 
we can use the BigQueryHelper.list_tables() method to list all the files in the hacker news dataset. 

In [3]:
# print a list of all the tables in the hacker_news dataset
hacker_news.list_tables()

['comments', 'full', 'full_201510', 'stories']

In [4]:
hacker_news.table_schema("full") 

,name,type,mode,description
0,title,STRING,NULLABLE,Story title
1,url,STRING,NULLABLE,Story url
2,text,STRING,NULLABLE,Story or comment text
3,dead,BOOLEAN,NULLABLE,Is dead?
4,by,STRING,NULLABLE,The username of the item's author.
5,score,INTEGER,NULLABLE,Story score
6,time,INTEGER,NULLABLE,Unix time
7,timestamp,TIMESTAMP,NULLABLE,Timestamp for the unix time
8,type,STRING,NULLABLE,"Type of details (comment, comment_ranking, pol..."
9,id,INTEGER,NULLABLE,The item's unique id.


Check the size of your query before you run it : 
BigQuery datasets are, true to their name, BIG. The biggest dataset we've got on Kaggle so far is 3 terabytes. Since the monthly quota for BigQuery queries is 5 terabytes, you can easily go past your 30-day quota by running just a couple of queries!

What's a query? A query is small piece of SQL code that specifies what data would you like to scan from a databases, and how much of that data you would like returned. (Note that your quota is on data scanned, not the amount of data returned.)

One way to help avoid this is to estimate how big your query will be before you actually execute it. You can do this with the BigQueryHelper.estimate_query_size() method. For the rest of this notebook, I'll be using an example query that finding the scores for every Hacker News post of the type "job". Let's see how much data it will scan if we actually ran it.

In [5]:
# this query looks in the full table in the hacker_news
# dataset, then gets the score column from every row where 
# the type column has "job" in it.
query = """SELECT score
            FROM `bigquery-public-data.hacker_news.full`
            WHERE type = "job" """

# check how big this query will be
hacker_news.estimate_query_size(query)

0.22514757234603167

Running this query will take around 225 MB. (The query size is returned in gigabytes.)

In [6]:

# only run this query if it's less than 100 MB
hacker_news.query_to_pandas_safe(query, max_gb_scanned=0.1)

Query cancelled; estimated size of 0.22514757234603167 exceeds limit of 0.1 GB


In [7]:
# check out the scores of job postings (if the 
# query is smaller than 1 gig)
#ob_post_scores = hacker_news.query_to_pandas_safe(query)

In [8]:
#Since this has returned a dataframe, we can work with it as
#we would any other dataframe. For example, we can get the mean of the column:

# average score for job posts
#job_post_scores.score.mean()